In [1]:
import pathlib
import logging
import threading
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
np.random.seed(0)
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
print(tf.__version__)
from sklearn.model_selection import StratifiedKFold
import time
porcentagemTeste = 0.1

2.7.0


In [2]:
dados = pd.read_csv('dados-Ambos-Tudo-mexidos.csv')
dados["QoE"] = dados["QoE"] + 3


#---------------------------------------------------------
#DROPANDO LINHAS COM USO DE CPU E RAM ALTO

dados = dados[dados['UsoCpu'] < 81]
dados = dados[dados['UsoRam'] < 81]
dados.sample(frac=1).reset_index(drop=True)

#DROPANDO LINHAS COM USO DE CPU E RAM ALTO
#---------------------------------------------------------


dados = dados.sample(frac=1).reset_index(drop=True)
#Salvando quantos jogadores temos e quantas instancias temos


#'NOME'
dadosFear = dados[dados['Jogo'] == 'FEAR']
dados = dados[dados['Jogo'] != 'FEAR']
dados.drop(['NOME','TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram','TerminoPartida','Jogo'], axis=1, inplace=True)
dados.dropna(inplace=True)
dadosFear.drop(['NOME','TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram','TerminoPartida','Jogo'], axis=1, inplace=True)
dadosFear.dropna(inplace=True)

In [3]:
def arredondaVetor(vetor):
    novoVetor = []
    for x in range(len(vetor)):
        if(vetor[x]>=0):
            floatTeste = vetor[x] - int(vetor[x])
            if(floatTeste>=0.5):
                novoVetor.append(int(vetor[x])+1)
            else:
                novoVetor.append(int(vetor[x]))
        else:
            valor = vetor[x]*(-1.0)
            floatTeste = valor - int(valor)
            if(floatTeste>=0.5):
                novoVetor.append((int(valor)+1)*-1)
            else:
                novoVetor.append(int(valor)*-1)
    return novoVetor

In [4]:
def porcentagemAcerto(vetor1,vetor2):
    tamanho = float(len(vetor1))
    acertos = float(0)
    for x in range(len(vetor1)):
        if(vetor1[x]==vetor2[x]):
            acertos = acertos + 1.0
    acertos = (acertos/tamanho)*100.0
    return acertos

In [5]:
def rodarModeloRFParam(baseDados,n_estimators,max_features,max_depth):
    global porcentagemTeste
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(baseDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X, y):
        rfr=RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,criterion='mse',bootstrap=True,max_depth=max_depth)
        rfr.fit(X[train],y[train])
        y_pred=rfr.predict(X[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(y_test,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
    
    return mse,acerto,model,X_test,y_test

In [6]:
def acharRandomForest():
    mse,acerto,model,X_test,y_test = rodarModeloRFParam(dados,1,1,1)
    estimators = 1
    features = 1
    depth = 1
    repeticoes = 10
    print("---------------------prourando melhores HiperParametros RandomForest-----------------------------")
    for n_estimators in range(1,101,5):
        for max_depth in range(5,21,1):
            for max_features in range(1,7,1):
              for ABA in range(repeticoes):
                mse1,acerto1,model1,X_test1,y_test1 = rodarModeloRFParam(dados,n_estimators,max_features,max_depth)
                if(mse1<mse):
                    mse = mse1
                    acerto = acerto1
                    model = model1
                    X_test = X_test1
                    y_test = y_test1
                    estimators = n_estimators
                    features = max_features
                    depth = max_depth
                print(".",end='')

    print("\nMenor MSE: "+str(round(mse,4))+" acerto: "+str(round(acerto,4)))
    print("n_estimators: "+str(estimators)+" max_features: "+str(features)+" max_depth: "+str(depth))
    print("Este modelo com todos os dados agora: ")
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos = porcentagemAcerto(y_test,test_predictions_round)
    print("Acerto com todos dados:",round(porcentagemAcertos,4))
    from sklearn.metrics import mean_squared_error
    mse1 = mean_squared_error(y_test,test_predictions_round)
    print("MSE com todos dados: "+str(mse1))

In [7]:
def rodarModeloMLPParam(baseDados,hidden_layer_sizes,max_iter,act,lr):
    if 'NOME' in baseDados.columns:
        baseDados.drop(['NOME'], axis=1, inplace=True)
        baseDados.dropna(inplace=True)
        
    global porcentagemTeste
    
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(baseDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X, y):
        
        from sklearn.neural_network import MLPRegressor
        rfr = MLPRegressor(learning_rate_init=lr,activation=act,random_state=1, max_iter=max_iter,hidden_layer_sizes = hidden_layer_sizes)
        rfr.fit(X[train],y[train])
        y_pred=rfr.predict(X[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(y_test,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
    
    return mse,acerto,model,X_test,y_test

In [8]:
def acharMLP():
    activation = ['identity','logistic','tanh','relu']
    act = activation[0]
    LR = [0.001,0.005,0.01,0.1]
    lr = LR[0]
    hidden_layers = [(32,32,),(32,32,32,),(32,32,32,32,),(64,64,),(64,64,64,),(64,64,64,64,)]
    hidden_layer = hidden_layers[0]
    max_iters = [250,500,750,1000]
    max_iter = max_iters[0]
    mse,acerto,model,X_test,y_test = rodarModeloMLPParam(dados,hidden_layer,max_iter,act,lr)
    repeticoes = 10
    print("---------------------prourando melhores HiperParametros MLP-----------------------------")
    for act1 in activation:
        for lr1 in LR:
            for hidden_layer1 in hidden_layers:
                for max_iter1 in max_iters:
                    for ABA in range(repeticoes):
                        mse1,acerto1,model1,X_test1,y_test1 = rodarModeloMLPParam(dados,hidden_layer1,max_iter1,act1,lr1)
                        if(mse1<mse):
                            mse = mse1
                            acerto = acerto1
                            model = model1
                            X_test = X_test1
                            y_test = y_test1
                            hidden_layer = hidden_layer1
                            max_iter = max_iter1
                            act = act1
                            lr = lr1
                        print(".",end='')

    print("\nMenor MSE: "+str(round(mse,4))+" acerto: "+str(round(acerto,4)))
    print("hidden_layer: "+str(hidden_layer)+" max_iter: "+str(max_iter)+" act: "+str(act)+" LR: "+str(lr))
    print("Este modelo com todos os dados agora: ")
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos = porcentagemAcerto(y_test,test_predictions_round)
    print("Acerto com todos dados:",round(porcentagemAcertos,4))
    from sklearn.metrics import mean_squared_error
    mse1 = mean_squared_error(y_test,test_predictions_round)
    print("MSE com todos dados: "+str(mse1))

In [9]:
def rodarModeloDTParam(baseDados,criterion,splitter,max_features,max_depth):

    global porcentagemTeste
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(baseDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X, y):
        
        from sklearn import tree
        rfr = tree.DecisionTreeRegressor(criterion=criterion,splitter=splitter,max_features=max_features,max_depth=max_depth)
        rfr.fit(X[train],y[train])
        y_pred=rfr.predict(X[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(y_test,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
    
    return mse,acerto,model,X_test,y_test

In [10]:
def acharDecisionTree():
    criterions = ['poisson','mae','mse','friedman_mse']
    criterion = criterions[0]
    splitters = ['best','random']
    splitter = splitters[0]
    max_featuress = [1,2,3,4,5,6]
    max_features = max_featuress[0]
    max_depths = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    max_depth = max_depths[0]
    mse,acerto,model,X_test,y_test = rodarModeloDTParam(dados,criterion,splitter,max_features,max_depth)
    repeticoes = 10
    print("---------------------prourando melhores HiperParametros Decision Tree-----------------------------")
    for criterion1 in criterions:
        for splitter1 in splitters:
            for max_features1 in max_featuress:
                for max_depth1 in max_depths:
                    for ABA in range(repeticoes):
                        mse1,acerto1,model1,X_test1,y_test1 = rodarModeloDTParam(dados,criterion1,splitter1,max_features1,max_depth1)
                        if(mse1<mse):
                            mse = mse1
                            acerto = acerto1
                            model = model1
                            X_test = X_test1
                            y_test = y_test1
                            criterion = criterion1
                            splitter = splitter1
                            max_features = max_features1
                            max_depth = max_depth1
                        print(".",end='')

    print("\nMenor MSE: "+str(round(mse,4))+" acerto: "+str(round(acerto,4)))
    print("criterion: "+str(criterion)+" splitter: "+str(splitter)+" max_features: "+str(max_features)+" max_depth: "+str(max_depth))
    print("Este modelo com todos os dados agora: ")
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos = porcentagemAcerto(y_test,test_predictions_round)
    print("Acerto com todos dados:",round(porcentagemAcertos,4))
    from sklearn.metrics import mean_squared_error
    mse1 = mean_squared_error(y_test,test_predictions_round)
    print("MSE com todos dados: "+str(mse1))

In [18]:
def rodarModeloADA(baseDados,n_estimators,learning_rate,loss):
        
    global porcentagemTeste
    
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(baseDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X, y):
        
        from sklearn.ensemble import AdaBoostRegressor
        from sklearn import tree
        #A base precisa utilizar os melhores parametros da decision tree 
        basee = tree.DecisionTreeRegressor(criterion='mse',splitter='best',max_features=4,max_depth=6)
        rfr = AdaBoostRegressor(base_estimator = basee,random_state=1, n_estimators=n_estimators,learning_rate=learning_rate,loss=loss)
        rfr.fit(X[train],y[train])
        y_pred=rfr.predict(X[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(y_test,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
    
    return mse,acerto,model,X_test,y_test

In [12]:
def acharAdaBoost():
    LR = [0.001,0.005,0.01,0.1]
    lr = LR[0]
    losss = ['square','linear','exponential']
    loss = losss[0]
    n_estimatorss = [25,50,75,100,125,150]
    n_estimators = n_estimatorss[0]
    mse,acerto,model,X_test,y_test = rodarModeloADA(dados,n_estimators,lr,loss)
    repeticoes = 10
    print("---------------------prourando melhores HiperParametros AdaBoost-----------------------------")
    for lr1 in LR:
        for loss1 in losss:
            for n_estimators1 in n_estimatorss:
                    for ABA in range(repeticoes):
                        mse1,acerto1,model1,X_test1,y_test1 = rodarModeloADA(dados,n_estimators1,lr1,loss1)
                        if(mse1<mse):
                            mse = mse1
                            acerto = acerto1
                            model = model1
                            X_test = X_test1
                            y_test = y_test1
                            n_estimators = n_estimators1
                            lr = lr1
                            loss = loss1
                        print(".",end='')

    print("\nMenor MSE: "+str(round(mse,4))+" acerto: "+str(round(acerto,4)))
    print("loss: "+str(loss)+" lr: "+str(lr)+" n_estimators: "+str(n_estimators))
    print("Este modelo com todos os dados agora: ")
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos = porcentagemAcerto(y_test,test_predictions_round)
    print("Acerto com todos dados:",round(porcentagemAcertos,4))
    from sklearn.metrics import mean_squared_error
    mse1 = mean_squared_error(y_test,test_predictions_round)
    print("MSE com todos dados: "+str(mse1))

In [13]:
acharDecisionTree()

---------------------prourando melhores HiperParametros Decision Tree-----------------------------


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [19]:
acharAdaBoost()

---------------------prourando melhores HiperParametros AdaBoost-----------------------------
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Menor MSE: 1.0644 acerto: 40.5941
loss: square lr: 0.005 n_estimators: 25
Este modelo com todos os dados agora: 
Acerto com todos dados: 39.604
MSE com todos dados: 1.1534653465346534


In [20]:
acharRandomForest()

---------------------prourando melhores HiperParametros RandomForest-----------------------------


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [21]:
acharMLP()

---------------------prourando melhores HiperParametros MLP-----------------------------
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multi

.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multi

.

...

.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Menor MSE: 1.0891 acerto: 37.6238
hidden_layer: (32, 32) max_iter: 250 act: logistic LR: 0.01
Este modelo com todos os dados agora: 
Acerto com todos dados: 34.6535
MSE com todos dados: 1.2475247524752475
